<a href="https://colab.research.google.com/github/GodJiLee/Dacon/blob/main/%EA%B5%AC%EB%82%B4%EC%8B%9D%EB%8B%B9%20%EC%8B%9D%EC%88%98%EC%9D%B8%EC%9B%90%20%EC%98%88%EC%B8%A1/80_8062307512_%EC%9B%90%ED%95%AB_%EA%B5%AC%EB%82%B4%EC%8B%9D%EB%8B%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **To Do List**
1. 요일 등 변수 팩터화 (0~4) 必 - get_dummies로 one-hot encoding 해보기 (O)
> https://teddylee777.github.io/scikit-learn/labelencoder-%EC%82%AC%EC%9A%A9%EB%B2%95   
> get_dummies, label_encoder, astype(‘category’).cat.codes   
> **get_dummies로 원-핫 인코딩 해주기로!**
2. LGBMRegressor 성능 최적화 방법 찾기
> 1. GridSearch로 파라미터 최적화   
> 2. 논문 찾기   
> 3. xgb, lgbm 결합하기
3. 공유코드 1, 3과도 결합지점 찾기 (1. 앙상블)
> https://www.kaggle.com/janged/xgb-lgb
4. 시각화 자료보고 인사이트 얻기 (O)
> 자기계발의 날, 공휴일 전후 처리 必
5. random_state 바꿔보기
6. LGBMRegressor에 공유코드 2,4 결합 data 학습
7. 주/연도 정보도 feature에 추가하기 (O)

# **모듈 임포트**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
# 경고 제어
warnings.filterwarnings(action = 'ignore') # 경고를 출력하지 않음

In [ ]:
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder # 수치형 자료가 아닌 자료에 대해 모델 학습을 위해 정수형으로 바꿔주는 기능
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor # Tree 기반 학습 알고리즘 (Gradient Boosting 프레임워크)

from gensim.models import Word2Vec
from gensim.models import KeyedVectors # 워드 벡터를 저장하고 관리하는 기능, 유사성 조회
from sklearn.manifold import TSNE # PCA 기반 차원 축소의 문제를 해결하기 위한 방법 (티스니)
from sklearn.decomposition import PCA

# **데이터 불러오기**

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_contest1/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_contest1/data/test.csv')

# **데이터 정제 (Dtype: object)**

### **일자, 요일은 라벨 인코딩 (숫자형으로 변환)**

In [ ]:
# 월, 일 인코딩
train['연도'] = pd.DatetimeIndex(train['일자']).year
test['연도'] = pd.DatetimeIndex(test['일자']).year

train['월'] = pd.DatetimeIndex(train['일자']).month
test['월'] = pd.DatetimeIndex(test['일자']).month

train['주'] = pd.DatetimeIndex(train['일자']).week
test['주'] = pd.DatetimeIndex(test['일자']).week

train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

# **데이터 정제 (아이디어)**
* 휴가자, 재택근무자는 중식/석식에 참여하지 않을 것이므로 본사 정원수에서 차감한다.
* 즉, 본사정원수 - 휴가자 - 재택근무자
* 식사참여율 = 식사계 / 식사가능자수

In [ ]:
train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

In [ ]:
# 식사가능자수 float -> int로 변경
train['식사가능자수'] = train['식사가능자수'].astype('int')
test['식사가능자수'] = test['식사가능자수'].astype('int')

In [ ]:
# 현본사소속재택근무자수 float -> int로 변경
train['현본사소속재택근무자수'] = train['현본사소속재택근무자수'].astype('int')
test['현본사소속재택근무자수'] = test['현본사소속재택근무자수'].astype('int')

In [ ]:
# 중식계/석식계 float -> int로 변경
train['중식계'] = train['중식계'].astype('int')
train['석식계'] = train['석식계'].astype('int')

In [ ]:
train['중식참여율'] = train['중식계'] / train['식사가능자수']
train['석식참여율'] = train['석식계'] / train['식사가능자수']

정제한 데이터를 정리하면,
### **요일 컬럼 처리**

### **Label Encoder로 범주형 자료 (요일, 요일(석식)) 수치형으로 바꾸기**

In [ ]:
# 수치형으로 바꾸지 않은 상태에서 encoder 적용
# cat_features = ['요일']

In [ ]:
# for feature in cat_features:
#     le = LabelEncoder()
#     le.fit(train[feature])
#     train[feature] = le.transform(train[feature])
#     test[feature] = le.transform(test[feature])

### **get_dummies로 원-핫 인코딩**
> 변수 간 관계성 삭제

In [ ]:
train = pd.get_dummies(train, columns = ['요일'])
test = pd.get_dummies(test, columns = ['요일'])

### **공휴일 전후 데이터 추가**
* 테스트 데이터에 설날 연휴, 어린이날 존재
* 연휴 전날 1로 인코딩 나머지 0

In [ ]:
# 새로운 column 생성
train['공휴일전후'] = 0
test['공휴일전후'] = 0

In [ ]:
# 한땀한땀 공휴일 지정..
train['공휴일전후'][17] = 1
train['공휴일전후'][3] = 1
train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후'][152] = 1
train['공휴일전후'][226] = 1
train['공휴일전후'][221] = 1
train['공휴일전후'][224] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후'][310] = 2
train['공휴일전후'][311] = 1
train['공휴일전후'][309] = 1
train['공휴일전후'][330] = 1
train['공휴일전후'][379] = 1
train['공휴일전후'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후'][470] = 1
train['공휴일전후'][502] = 2
# train['공휴일전후'][501] = 1
# train['공휴일전후'][511] = 1
train['공휴일전후'][565] = 1
train['공휴일전후'][623] = 1
train['공휴일전후'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후'][705] = 1
# train['공휴일전후'][707] = 1
train['공휴일전후'][709] = 1
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후'][815] = 1
train['공휴일전후'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후'][950] = 1
train['공휴일전후'][951] = 1
train['공휴일전후'][953] = 1
train['공휴일전후'][954] = 1
train['공휴일전후'][955] = 1
train['공휴일전후'][971] = 2
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후'][1038] = 1
train['공휴일전후'][1099] = 1
train['공휴일전후'][1129] = 1
# train['공휴일전후'][1128] = 1
train['공휴일전후'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후'][10] = 2
test['공휴일전후'][20] = 1

**원-핫 인코딩**

In [ ]:
# 변수간 관계성 삭제 (더미화, 가변수화)
train = pd.get_dummies(train, columns = ['공휴일전후'])
test = pd.get_dummies(test, columns = ['공휴일전후'])

In [ ]:
test['공휴일전후_0'][20] = 1
test['공휴일전후_1'][20] = 0

### **필요없는 features 버리기 (drop)**

In [ ]:
drops = ['일자', '연도', '일', '조식메뉴', '중식메뉴', '석식메뉴']

train = train.drop(drops, axis = 1) # axis = 1은 열을 의미 (열삭제)
test = test.drop(drops, axis = 1)

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   본사정원수           1205 non-null   int64  
 1   본사휴가자수          1205 non-null   int64  
 2   본사출장자수          1205 non-null   int64  
 3   본사시간외근무명령서승인건수  1205 non-null   int64  
 4   현본사소속재택근무자수     1205 non-null   int64  
 5   중식계             1205 non-null   int64  
 6   석식계             1205 non-null   int64  
 7   월               1205 non-null   int64  
 8   주               1205 non-null   int64  
 9   식사가능자수          1205 non-null   int64  
 10  중식참여율           1205 non-null   float64
 11  석식참여율           1205 non-null   float64
 12  요일_금            1205 non-null   uint8  
 13  요일_목            1205 non-null   uint8  
 14  요일_수            1205 non-null   uint8  
 15  요일_월            1205 non-null   uint8  
 16  요일_화            1205 non-null   uint8  
 17  공휴일전후_0         1205 non-null   u

In [ ]:
test.head()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,월,주,식사가능자수,요일_금,요일_목,요일_수,요일_월,요일_화,공휴일전후_0,공휴일전후_1,공휴일전후_2
0,2983,88,182,5,358,1,4,2537,0,0,1,0,0,1,0,0
1,2983,104,212,409,348,1,4,2531,0,1,0,0,0,1,0,0
2,2983,270,249,0,294,1,4,2419,1,0,0,0,0,1,0,0
3,2924,108,154,538,322,2,5,2494,0,0,0,1,0,1,0,0
4,2924,62,186,455,314,2,5,2548,0,0,0,0,1,1,0,0


In [ ]:
train.head()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,월,주,식사가능자수,중식참여율,석식참여율,요일_금,요일_목,요일_수,요일_월,요일_화,공휴일전후_0,공휴일전후_1,공휴일전후_2
0,2601,50,150,238,0,1039,331,2,5,2551,0.407291,0.129753,0,0,0,1,0,1,0,0
1,2601,50,173,319,0,867,560,2,5,2551,0.339867,0.219522,0,0,0,0,1,1,0,0
2,2601,56,180,111,0,1017,573,2,5,2545,0.399607,0.225147,0,0,1,0,0,1,0,0
3,2601,104,220,355,0,978,525,2,5,2497,0.391670,0.210252,0,1,0,0,0,0,1,0
4,2601,278,181,34,0,925,330,2,5,2323,0.398192,0.142058,1,0,0,0,0,1,0,0


### **모델에 사용할 데이터 생성**

In [ ]:
# features는 학습시킬 특성값들, labels는 추론할 대상값들
# 본사정원수 제외 (선형성 고려)
features = ['본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수', '월', '주', '식사가능자수', '요일_월', '요일_화', '요일_수', '요일_목', '요일_금', '공휴일전후_0', '공휴일전후_1', '공휴일전후_2']
labels = ['중식계', '석식계'] # '중식참여율', '석식참여율'

In [ ]:
# lunch

# train
train_x = train[features]
train_y_lunch = train['중식계']
train_y_dinner = train['석식계']
# test
test_x = test[features]

In [ ]:
train_x.head(2)

,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,월,주,식사가능자수,요일_월,요일_화,요일_수,요일_목,요일_금,공휴일전후_0,공휴일전후_1,공휴일전후_2
0,50,150,238,0,2,5,2551,1,0,0,0,0,1,0,0
1,50,173,319,0,2,5,2551,0,1,0,0,0,1,0,0


In [ ]:
train_y_lunch.head(2)

0    1039
1     867
Name: 중식계, dtype: int64

In [ ]:
train_y_dinner.head(2)

0    331
1    560
Name: 석식계, dtype: int64

In [ ]:
test_x.head(2)

,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,월,주,식사가능자수,요일_월,요일_화,요일_수,요일_목,요일_금,공휴일전후_0,공휴일전후_1,공휴일전후_2
0,88,182,5,358,1,4,2537,0,0,1,0,0,1,0,0
1,104,212,409,348,1,4,2531,0,0,0,1,0,1,0,0


# **Modeling**

### **LGBMRegressor**

In [ ]:
# Simple LGBM Regressor w/o tuning
ln_model = LGBMRegressor()
ln_model.fit(train_x, train_y_lunch)

dn_model = LGBMRegressor()
dn_model.fit(train_x, train_y_dinner)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
# Prediction
pred_lunch = ln_model.predict(test_x)
pred_dinner = dn_model.predict(test_x)

# **Inference**

In [ ]:
# Inference
test_pred_lunch = ln_model.predict(test_x)
test_pred_dinner = dn_model.predict(test_x)

submission_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_contest1/data/sample_submission.csv')

In [ ]:
submission_df.head()

,일자,중식계,석식계
0,2021-01-27,0,0
1,2021-01-28,0,0
2,2021-01-29,0,0
3,2021-02-01,0,0
4,2021-02-02,0,0


In [ ]:
submission_df['중식계'] = test_pred_lunch
submission_df['석식계'] = test_pred_dinner

In [ ]:
# Save

submission_df.to_csv('submission_원핫_공휴일.csv', index = False)

### **파일 제출**

In [ ]:
!pip install dacon_submit_api-0.0.4-py3-none-any.whl

Processing ./dacon_submit_api-0.0.4-py3-none-any.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/content/dacon_submit_api-0.0.4-py3-none-any.whl'



In [ ]:
from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'파일경로', 
'3c402589c611d3aef251b15c37459c9b15abf6f4c5edbbbe98e8a6491c8703cf', 
'대회ID', 
'팀이름', 
'submission 메모 내용' )